## Day 24: Arithmetic Logic Unit

[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.org/github/mazharenko/AoC-2021/tree/HEAD/notebooks/day24/puzzle.ipynb)

In [ ]:
#load "../common.fsx"

#r "nuget:Farkle, 6.3.2"
open Farkle
open Farkle.Builder

type Registers = { X : int ref; Y : int ref; Z : int ref; W : int ref}

let private number = Terminals.genericSigned<int> "Number"
let private register = "register" ||= [
    !& "x" => (fun () -> fun registers -> registers.X)
    !& "y" => (fun () -> fun registers -> registers.Y)
    !& "z" => (fun () -> fun registers -> registers.Z)
    !& "w" => (fun () -> fun registers -> registers.W)
]
let private registerOrValue = "value" ||= [
    !@ register => (fun registerFun -> fun registers -> !registerFun(registers))
    !@ number => (fun value -> fun _ -> value)
]

let private command = "command" ||= [
    !& "inp w" => (fun () -> fun registers -> ()) // gonna set W manually
    !& "add" .>>. register .>>. registerOrValue => (
        fun reg value -> 
            fun registers -> 
                let register = reg(registers)
                let value = value(registers)
                register := !register + value
    )
    !& "mul" .>>. register .>>. registerOrValue => (
        fun reg value -> 
            fun registers -> 
                let register = reg(registers)
                let value = value(registers)
                register := !register * value
    )
    !& "div" .>>. register .>>. registerOrValue => (
        fun reg value -> 
            fun registers -> 
                let register = reg(registers)
                let value = value(registers)
                register := !register / value
    )
    !& "mod" .>>. register .>>. registerOrValue => (
        fun reg value -> 
            fun registers -> 
                let register = reg(registers)
                let value = value(registers)
                register := !register % value
    )
    !& "eql" .>>. register .>>. registerOrValue => (
        fun reg value -> 
            fun registers -> 
                let register = reg(registers)
                let value = value(registers)
                register := if !register = value then 1 else 0
    )
]

let commandParser = RuntimeFarkle.build command


Installed Packages Farkle, 6.3.2

In [ ]:
#!value --name inputRaw --from-file ./data

In [ ]:
#!share inputRaw --from value

type Subprogram = { Commands: (Registers -> unit) list }

let subprograms =
    readLines inputRaw
    |> Seq.chunkBySize 18
    |> Seq.map (fun subprogram -> subprogram |> Seq.map (RuntimeFarkle.parseString commandParser) |> Seq.map Result.failIfError |> List.ofSeq)
    |> Seq.map (fun subprogram -> {Commands = subprogram})
    |> Seq.toArray

let execute registers subprogram =
    subprogram.Commands |> List.iter (fun command -> command registers)
    registers

In [ ]:
type State = { Registers : Registers }

In [ ]:
open System.Collections.Generic

let memoize f keyf =
    let cache = Dictionary<_, _>()    
    fun x ->
        let key = keyf x
        match cache.TryGetValue(key) with
        | true, value -> 
            value
        | _ ->
            let value = f x
            cache.Add(key, None)
            value

In [ ]:

let rec findResults = 
    memoize (fun (index, subprograms, state) -> 
        if (state.Registers.Z.Value > pown 26 4) then None
        else 
            match Array.tryItem index subprograms with 
            | None ->
                if (state.Registers.Z.Value = 0) then 
                    Option<list<list<int>>>.Some [[]]
                else None
            | Some currentSubprogram -> 
                let g = 
                    [1..9]
                    |> List.choose (fun w -> 
                        let newRegisters = { X = ref !state.Registers.X; Y = ref !state.Registers.Y; Z = ref !state.Registers.Z; W = ref w }
                        let newState = { Registers = execute newRegisters currentSubprogram }
                        findResults (index+1, subprograms, newState)
                        |> Option.map (fun l -> l |> List.map(fun ll -> w::ll))
                    ) |> List.collect id
                Some g

    ) (fun (index, subprograms, {Registers = {Z = z; W = w}}) -> index, !z, !w)
    
let allInputs = findResults (0, subprograms, { Registers = {X = ref 0; Y = ref 0; Z = ref 0; W = ref 0}})

String.Join("", allInputs.Value |> List.max) |> display
String.Join("", allInputs.Value |> List.min) |> display
        

53621111481319

13621111481315